# Paper Figures
1. Map of regions
2. `NPP` predictability heatmap
3. `SIE` predictability heatmap
4. "Propagation" of predictability (`SIE`, `MLD`, `SFC_IRR`, `NPP`, and maybe `SFC_CHL`, `SFC_BIOMASS`) 
5. `SIE` predicting `NPP` cross correlation
6. `SFC_IRR` predicting `NPP` cross correlation

In [11]:
import xarray as xr
import numpy as np
import scipy.stats as stats
import pickle as pkl
import matplotlib.pyplot as plt
import matplotlib as mpl
import cmocean.cm as cmo

In [2]:
mpl.rcParams['figure.dpi'] = 300

%config InlineBackend.figure_formats = ['pdf']

In [3]:
def open_metric(var, reg, metric, timescale='monthly', ens_type=''):
    
    writedir = '/home/bbuchovecky/storage/so_predict_derived/'
    
    if metric == 'clim':
        subdir = 'CTRL/'+var.upper()+'/'
        filename = var.lower()+'_ts_'+reg+'_'+metric+'.nc'
    
    if metric == 'anom' or metric == 'mean' or (metric == 'var' and timescale == 'monthly'):
        subdir = 'CTRL/'+var.upper()+'/'
        filename = var.lower()+'_ts_'+reg+'_'+timescale+'_'+metric+'.nc'
    
    if metric.lower() == 'ppp':
        subdir = 'PPP/'+var.upper()+'/'
        if ens_type != '':
            ens_type += '_'
        filename = var.lower()+'_ts_'+reg+'_'+timescale+'_'+ens_type+'ppp.nc'
        
    return xr.open_dataset(writedir+subdir+filename)

def get_plotting_labels():
    with open('/home/bbuchovecky/storage/so_predict_derived/plotting_dicts.pkl','rb') as handle:
        plotting_dicts = pkl.load(handle)
    
    reg_names = plotting_dicts['reg_names']
    var_abbrv_names = plotting_dicts['var_abbrv_names']
    abbrv_month_names = plotting_dicts['abbrv_month_names']
    month_letters = plotting_dicts['month_letters']
    
    return reg_names, var_abbrv_names, abbrv_month_names, month_letters

## Regional Map

## Predictability Heatmap 

In [4]:
def ppp_simple_heatmap(
    var, vmin=0, threshold=0.183, hatch='//', cmap=cmo.matter, figsize=(12,8)):

    file = open('/home/bbuchovecky/storage/so_predict_derived/plotting_dicts.pkl','rb')
    plotting_dicts = pkl.load(file)
    file.close()

    reg_names = plotting_dicts['reg_names']
    var_su_names = plotting_dicts['var_su_names']
    var_lu_names = plotting_dicts['var_lu_names']
    var_ll_names = plotting_dicts['var_ll_names']
    abbrv_month_names = plotting_dicts['abbrv_month_names']

    ppp = open_metric(var, 'so', 'ppp')
    regions = ['Weddell', 'Indian', 'WestPacific', 'SouthernOcean', 'Ross', 'AmundBell']

    fig,ax = plt.subplots(2, 3, sharex=True, sharey=True, figsize=figsize)
    
    subplots = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)']

    for (ireg,reg) in zip(range(len(regions)), regions):
        ppp_matrix = np.zeros((10,12))

        for m in range(ppp['nT'].size):
            ppp_matrix[int(m/12),m%12] = ppp[reg][m]

        ppp_matrix = ppp_matrix.T

        im = ax[int(ireg/3),ireg%3].pcolormesh(ppp_matrix, vmin=vmin, vmax=1.0, cmap=cmap, shading='auto')

        ax[int(ireg/3),ireg%3].set_title(subplots[ireg]+' '+reg_names[reg], fontweight='bold', loc='left')

        ax[int(ireg/3),ireg%3].set_yticks(np.arange(ppp_matrix.shape[0]) + 0.5)
        ax[int(ireg/3),ireg%3].set_yticklabels(abbrv_month_names, fontsize='large')

        ax[int(ireg/3),ireg%3].set_xticks(np.arange(ppp_matrix.shape[1]) + 0.5)
        ax[int(ireg/3),ireg%3].set_xticklabels(np.arange(1,11,1), fontsize='large')
        ax[int(ireg/3),ireg%3].set_aspect('auto')

        if int(ireg/3) == 1:
            ax[int(ireg/3),ireg%3].set_xlabel('Year', fontsize='large')

        if hatch:
            masked = np.ma.masked_array(ppp_matrix, mask=np.where(ppp_matrix > threshold, 0, 1))
            masked_im = ax[int(ireg/3),ireg%3].pcolor(masked, hatch=hatch, alpha=0)

#     plt.suptitle(var_su_names[var], x=0.402, ha='left', fontsize=16)
    fig.tight_layout()

    cb = fig.colorbar(im, ax=ax.ravel().tolist())
    cb.ax.fill_between([-0.1,1.1], [1.0,1.0], [threshold,threshold], hatch=hatch, alpha=0)
#     cb.ax.plot([-0.1,1.1], [threshold,threshold], color='k', linewidth=4)
#     cb.ax.text(1.3, threshold-0.015, str(threshold))
    cb.ax.set_ylabel('PPP', fontsize='large')
    cb.ax.text(0.5, vmin-0.03, 'No skill', ha='center', va='center', fontsize='large')
    cb.ax.text(0.5, 1.0+0.03, 'Perfect skill', ha='center', va='center', fontsize='large')

In [5]:
ppp_simple_heatmap('npp', threshold=1)

<Figure size 3600x2400 with 7 Axes>

In [6]:
ppp_simple_heatmap('sie', threshold=1)

<Figure size 3600x2400 with 7 Axes>

## "Propagation" of Predictability

In [42]:
def ppp_var_heatmap(
    variables, reg, num_months=24, vmin=-0.2, threshold=0.183, hatch='//', yaxis_steps=2, cmap=cmo.matter, figsize=(8,7)):
    
    file = open('/home/bbuchovecky/storage/so_predict_derived/plotting_dicts.pkl','rb')
    plotting_dicts = pkl.load(file)
    file.close()

    reg_names = plotting_dicts['reg_names']
    var_su_names = {'npp': 'NPP',
                     'mld': 'MLD',
                     'sie': 'SIE',
                     'sst': 'SST',
                     'sss': 'SSS',
                     'cn_inv': 'SIC',
                     'pco2surf': 'Surface pCO$_2$',
                     'siv': 'SIV',
                     'sfc_fed': 'Iron',
                     'sfc_irr': 'Irradiance',
                     'sfc_biomass': 'Biomass',
                     'sfc_chl': 'Chl $a$'}

#     regions = ['SouthernOcean', 'Weddell', 'Indian', 'WestPacific', 'Ross', 'AmundBell']
    regions = ['Weddell', 'Indian', 'WestPacific', 'SouthernOcean', 'Ross', 'AmundBell']

    variable_names = []
    
    if reg.lower() == 'all':
        
        fig,ax = plt.subplots(2, 3, sharex=True, figsize=figsize)
        
        subplots = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)']

        for (ireg,reg) in zip(range(len(regions)), regions):
            ppp_matrix = np.zeros((len(variables),num_months))

            for (i,var) in zip(range(len(variables)), variables):
                ppp = open_metric(var, 'so', 'ppp')
                ppp_matrix[i] = ppp[reg][0:num_months]
                if ireg == 0:
                    variable_names.append(var_su_names[var])

            ppp_matrix = ppp_matrix.T

            im = ax[int(ireg/3),ireg%3].pcolormesh(ppp_matrix, vmin=vmin, vmax=1.0, cmap=cmap, shading='auto')

            ax[int(ireg/3),ireg%3].set_yticks(np.arange(0,ppp_matrix.shape[0],yaxis_steps) + 0.5)
#             mult = int(num_months/12)+1
#             ylabels = (abbrv_month_names*mult)[0:num_months:yaxis_steps]
            
            ax[int(ireg/3),ireg%3].set_yticklabels(np.arange(0,ppp_matrix.shape[0],yaxis_steps), fontsize='large')
            
#             for (i,yr) in zip(range(len(ylabels)), range(0,num_months,yaxis_steps)):
#                 if ylabels[i] == 'Jan':
# #                     ylabels[i] = ylabels[i] + ' ' + str(yr//12 + 1)
#                     ylabels[i] = str(yr//12 + 1)+'   '+ylabels[i]
                
#             ax[int(ireg/3),ireg%3].set_yticklabels(ylabels)
            
            if ireg%3 == 0:
                ax[int(ireg/3),ireg%3].set_ylabel('Lead (months)', fontsize='large')

            ax[int(ireg/3),ireg%3].set_xticks(np.arange(ppp_matrix.shape[1]) + 0.5)
            ax[int(ireg/3),ireg%3].set_xticklabels(variable_names, fontsize='large')
            plt.setp(ax[int(ireg/3),ireg%3].get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor", fontsize='large')
            
            ax[int(ireg/3),ireg%3].set_title(subplots[ireg]+' '+reg_names[reg], fontweight='bold', loc='left')

    
            if hatch:
                masked = np.ma.masked_array(ppp_matrix, mask=np.where(ppp_matrix > threshold, 0, 1))
                masked_im = ax[int(ireg/3),ireg%3].pcolor(masked, hatch=hatch, alpha=0)

        fig.tight_layout()

        cb = fig.colorbar(im, ax=ax.ravel().tolist())
#         cb.ax.plot([-0.1,1.1], [threshold,threshold], color='k', linewidth=4)
#         cb.ax.plot([-0.1,1.1], [0.183,0.183], color='k', ls='-', linewidth=1)
        cb.ax.fill_between([-0.1,1.1], [1.0,1.0], [threshold,threshold], hatch=hatch, alpha=0)
#         cb.ax.text(1.3, threshold-0.015, str(threshold))
        
        cb.ax.text(0.5, vmin-0.03, 'No skill', ha='center', va='center', fontsize='large')
        cb.ax.text(0.5, 1.0+0.03, 'Perfect skill', ha='center', va='center', fontsize='large')

        return fig,ax
            
#     ## single plots
#     if reg.lower() == 'global':
#         subreg = 'global'
#         reg = 'Global'

#     if reg.lower() != 'global' and reg.lower() != 'all':
#         subreg = 'so'

#     ppp_matrix = np.zeros((len(variables),num_months))

#     for (i,var) in zip(range(len(variables)), variables):
#         ppp = open_metric(var, subreg, 'ppp')
#         ppp_matrix[i] = ppp[reg][0:num_months]
#         variable_names.append(var_su_names[var])

#     ppp_matrix = ppp_matrix.T

#     fig,ax = plt.subplots(figsize=figsize)

#     im = ax.pcolormesh(ppp_matrix, vmin=vmin, vmax=1.0, cmap=cmap, shading='auto')
    
#     ax.set_yticks(np.arange(0,ppp_matrix.shape[0],yaxis_steps) + 0.5)
# #     mult = int(num_months/12)+1
# #     ax.set_yticklabels((abbrv_month_names*mult)[0:num_months:yaxis_steps])
# #     ax.set_ylabel('Forecast month')
#     ax.set_yticklabels(np.arange(0,ppp_matrix.shape[0]))
#     ax.set_ylabel('Lead (months)')
    
#     ax.set_xticks(np.arange(ppp_matrix.shape[1]) + 0.5)
#     ax.set_xticklabels(variable_names)

#     plt.setp(ax.get_xticklabels(), rotation=45, ha='right', rotation_mode='anchor', fontsize='large')

#     cb = fig.colorbar(im, label='PPP')
    
#     if hatch:
#         masked = np.ma.masked_array(ppp_matrix, mask=np.where(ppp_matrix > threshold, 0, 1))
#         masked_im = ax.pcolor(masked, hatch=hatch, alpha=0)
#         cb.ax.plot([-0.1,1], [threshold,threshold], color='k', linewidth=4)
        
#         cb.ax.text(1.45, threshold, 'Hatch threshold ('+str(threshold)+')')
        
#         cb.ax.text(0.5, vmin-0.05, 'No skill', ha='center')
#         cb.ax.text(0.5, 1.0+0.03, 'Perfect skill', ha='center')

#     fig.suptitle(reg_names[reg]+' - PPP', fontsize=16)
#     fig.tight_layout()
    
    return fig,ax

In [43]:
variables = ['sie', 'mld', 'sfc_irr', 'npp']
ppp_var_heatmap(variables, 'all', num_months=24, figsize=(10,8), vmin=0, threshold=1);

<Figure size 3000x2400 with 7 Axes>

In [44]:
variables = ['sie', 'mld', 'sfc_irr', 'npp', 'sfc_chl', 'sfc_biomass']
ppp_var_heatmap(variables, 'all', num_months=24, figsize=(13,8), vmin=0, threshold=1);

<Figure size 3900x2400 with 7 Axes>

## Cross Correlation

In [23]:
## predictor --> the field doing the predicting (e.g., SIE or SFC_IRR)
## predictand --> the field being predicted (e.g., NPP)
def compute_lagged_xcorr(predictor, predictand, maxlag):
    ## convert DataArray to NumPy array
    predictor = predictor.values
    predictand = predictand.values
    
    assert predictor.size == predictand.size, 'predictor and predictand have different lengths'
    N = predictor.size
    
    ## rows are different init months, cols are different lags
    r_matrix = np.zeros((abs(maxlag)+1, 12))
    p_matrix = np.zeros((abs(maxlag)+1, 12))

    init_matrix = np.zeros((abs(maxlag)+1, 12))
    lag_matrix = np.zeros((abs(maxlag)+1, 12))
    
    lag_values = np.arange(maxlag, 1)
    for (it,init) in enumerate(range(0,12)):
        for (ig,lag) in enumerate(lag_values):   
            trim = 12*((abs(lag)-1)//12+1)
            init_matrix[ig][it] = init
            lag_matrix[ig][it] = lag
            
            if lag == 0:
                tmp_predictand = predictand[init:N:12]
                tmp_predictor = predictor[init:N:12]
                
            else:
                tmp_predictand = predictand[init+trim:N:12]
                tmp_predictor = predictor[init+trim+lag:N-trim+init:12]
            
            r_matrix[ig][it], p_matrix[ig][it] = stats.pearsonr(tmp_predictand, tmp_predictor)
            
    return r_matrix, p_matrix, init_matrix, lag_matrix

def plot_regional_lagged_xcorr(predictor, predictand, maxlag=-24, threshold=0.75, hatch='//', figsize=(12,8)):
    reg_names, var_su_names, abbrv_month_names, month_letters = get_plotting_labels()
    region_list = ['Weddell', 'Indian', 'WestPacific', 'SouthernOcean', 'Ross', 'AmundBell']

    fig,ax = plt.subplots(2, 3, figsize=figsize)
    
    subplots = ['(a)', '(b)', '(c)', '(d)', '(e)', '(f)']
    
    for (ireg,reg) in enumerate(region_list):  
        ts_predictor = open_metric(predictor, 'so', 'anom')[reg]
        ts_predictand = open_metric(predictand, 'so', 'anom')[reg]

        r, p, inits, lags = compute_lagged_xcorr(ts_predictor, ts_predictand, maxlag=maxlag)
    
        ## transform r so that lead=0 is the bottom row and lead=maxlag is top row
        numlags = r.shape[0]
        r_transformed = np.empty(shape=r.shape)
        for i in range(numlags):
            r_transformed[numlags-i-1] = r[i]
        r = r_transformed
        
        im = ax[int(ireg/3),ireg%3].pcolormesh(r, vmin=-1.0, vmax=1.0, cmap=cmo.balance, edgecolor='k', linewidth=0.01)
        
        if hatch:
            masked = np.ma.masked_array(r, mask=np.where(abs(r) > threshold, 0, 1))
            masked_im = ax[int(ireg/3),ireg%3].pcolor(masked, hatch=hatch, alpha=0)
        
        ax[int(ireg/3),ireg%3].set_yticks((np.arange(r.shape[0]) + 0.5)[::4])
        ax[int(ireg/3),ireg%3].set_yticklabels(np.arange(0,-maxlag+1)[::4], fontsize='large')
        ax[int(ireg/3),ireg%3].set_xticks((np.arange(r.shape[1]) + 0.5))
        ax[int(ireg/3),ireg%3].set_xticklabels(month_letters, fontsize='large')
        ax[int(ireg/3),ireg%3].set_title(subplots[ireg]+' '+reg_names[reg], fontweight='bold', loc='left')
        
        if ireg%3 == 0:
            ax[int(ireg/3),ireg%3].set_ylabel('Lead (month)', fontsize='large')
#         if ireg//3 == 1:
#             ax[int(ireg/3),ireg%3].set_xlabel('Init month')
    

#     fig.suptitle(var_su_names[predictor]+' predicting '+var_su_names[predictand], x=0, ha='left', fontweight='bold', fontsize='x-large')
    fig.tight_layout()
    
    cb = fig.colorbar(im, ax=ax.ravel().tolist())
    cb.set_label('Correlation coefficient', fontsize='large')
    
    if hatch:
        cb.ax.plot([-1,1], [threshold,threshold], color='k')
        cb.ax.plot([-1,1], [-threshold,-threshold], color='k')
        cb.ax.fill_between([-1,1], [1,1], [threshold,threshold], hatch=hatch, alpha=0)
        cb.ax.fill_between([-1,1], [-threshold,-threshold], [-1,-1], hatch=hatch, alpha=0)

In [25]:
predictor = 'sie'
predictand = 'npp'
plot_regional_lagged_xcorr(predictor, predictand, maxlag=-16, hatch=False, figsize=(10,7));

<Figure size 3000x2100 with 7 Axes>

In [26]:
predictor = 'sfc_irr'
predictand = 'npp'
plot_regional_lagged_xcorr(predictor, predictand, maxlag=-16, hatch=False, figsize=(10,7));

<Figure size 3000x2100 with 7 Axes>